<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!pip install libtorrent
!apt install 7zip
!pip install dropbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 74.9 MB/s  0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  7zip

In [ ]:
import sys
import os, subprocess, glob
import time

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn

from pathlib import Path
import libtorrent as lt

import dropbox
from dropbox.files import WriteMode
from dropbox.exceptions import ApiError, AuthError

import concurrent.futures

In [ ]:
link = input("Torrent/Magnet link: \n") # PASTE TORRENT/MAGNET LINK HERE
TOKEN = str(Path("dbx.tkn").read_text().rstrip().strip())

Torrent/Magnet link: 
magnet:?xt=urn:btih:dad013f699d82b30fcee3075b6056966ad4d7913&dn=%5BErai-raws%5D%20City%20the%20Animation%20-%2004%20%5B1080p%20AMZN%20WEB-DL%20AVC%20EAC3%5D%5BMultiSub%5D%5B9A6A32E7%5D&tr=http%3A%2F%2Fnyaa.tracker.wf%3A7777%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce


In [ ]:
save_path = Path("/content/Torrent")
save_path.mkdir(parents=True, exist_ok=True)

def clear_console():
    cmd = "cls" if os.name == "nt" else "clear"
    subprocess.call(cmd, shell=True)

ses = lt.session()
try:
  handle = lt.parse_magnet_uri(link)
except Exception as e:
  print("Failed to parse magnet URI: ", e)
  sys.exit(1)

handle.save_path = str(save_path)
atp = ses.add_torrent(handle)
s = atp.status()

while not s.has_metadata:
  time.sleep(1/1000)
  s = atp.status()

with Progress(
        TextColumn("[bold]{task.fields[filename]}"),
        BarColumn(),
        TaskProgressColumn(),
        TimeRemainingColumn(),
        TextColumn("{task.fields[speed]}"),
        ) \
as progress:
  task = progress.add_task(
            "download",
            total=100,
            filename=atp.status().name,
            speed="0.0 kB/s",
        )

  while not s.is_seeding:
    s = atp.status()

    progress.update(
                task,
                completed=s.progress * 100,
                speed=f"{s.download_rate/1000:.1f} kB/s",
            )

  progress.update(
                task,
                completed=s.progress * 100,
                speed=f"{s.download_rate/1000:.1f} kB/s",
            )

Output()

In [ ]:
subprocess.run(["7za", "a", "-v1024m", "/content/Torrent.zip", "/content/Torrent/"])

CompletedProcess(args=['7za', 'a', '-v1024m', '/content/Torrent.zip', '/content/Torrent/'], returncode=2)

In [ ]:
# def upload(
#     access_token,
#     file_path,
#     target_path,
#     timeout=900,
#     chunk_size=4 * 1024 * 1024,
# ):
#     dbx = dropbox.Dropbox(access_token, timeout=timeout)
#     with open(file_path, "rb") as f:
#         file_size = os.path.getsize(file_path)
#         if file_size <= chunk_size:
#             print(dbx.files_upload(f.read(), target_path))
#         else:
#             with tqdm(total=file_size, desc="Uploaded") as pbar:
#                 upload_session_start_result = dbx.files_upload_session_start(
#                     f.read(chunk_size)
#                 )
#                 pbar.update(chunk_size)
#                 cursor = dropbox.files.UploadSessionCursor(
#                     session_id=upload_session_start_result.session_id,
#                     offset=f.tell(),
#                 )
#                 commit = dropbox.files.CommitInfo(path=target_path)
#                 while f.tell() < file_size:
#                     if (file_size - f.tell()) <= chunk_size:
#                         print(
#                             dbx.files_upload_session_finish(
#                                 f.read(chunk_size), cursor, commit
#                             )
#                         )
#                     else:
#                         dbx.files_upload_session_append(
#                             f.read(chunk_size),
#                             cursor.session_id,
#                             cursor.offset,
#                         )
#                         cursor.offset = f.tell()
#                     pbar.update(chunk_size)

torrents = list(Path("/content/").glob("Torrent.zip.*"))
# upload(TOKEN, torrents[1].as_posix(), "/Torrent/" + torrents[1].name)
# for torrent in torrents:


# run uploads in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = [
        executor.submit(upload, TOKEN, torrent.as_posix(), "/Torrent/" + torrent.name)
        for torrent in torrents
    ]
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print("Upload failed:", e)

Uploaded: 931135488it [03:34, 4341456.09it/s]

FileMetadata(client_modified=datetime.datetime(2025, 10, 6, 6, 35, 57), content_hash='7da9e45bf034cf26c2e008db41876fc4150e4e0eef238532d2e7ea92cc9631cd', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:uarvdFOTHrAAAAAAAAAAHg', is_downloadable=True, media_info=NOT_SET, name='Torrent.zip.001', parent_shared_folder_id=NOT_SET, path_display='/Torrent/Torrent.zip.001', path_lower='/torrent/torrent.zip.001', preview_url=NOT_SET, property_groups=NOT_SET, rev='0164077aa05e6c60000000300d37ac1', server_modified=datetime.datetime(2025, 10, 6, 6, 35, 58), sharing_info=NOT_SET, size=928419183, symlink_info=NOT_SET)
